In [1]:
# !pip install -q -U  langchain-nomic 



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
# 基本配置
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv

load_dotenv(override=True)

qw_llm_openai = ChatOpenAI(
    openai_api_base=os.getenv('DASHSCOPE_API_BASE'),
    openai_api_key=os.getenv('DASHSCOPE_API_KEY'),
    model_name="qwen2-1.5b-instruct",
    temperature=0,
    streaming=True,
)

ms_llm_openai = ChatOpenAI(
    openai_api_base=os.getenv('MOONSHOT_API_BASE'),
    openai_api_key=os.getenv('MOONSHOT_API_KEY'),
    model_name="moonshot-v1-8k",
    temperature=0,
    streaming=True,
)

cf_llm_openai = ChatOpenAI(
    openai_api_base=os.getenv('CF_API_BASE'),
    openai_api_key=os.getenv('CF_API_TOKEN'),
    # model_name="@cf/meta/llama-3-8b-instruct",
    model_name="@cf/meta/llama-3.1-8b-instruct",
    temperature=0,
    streaming=True,
)

groq_llm_openai = ChatOpenAI(
    openai_api_base=os.getenv('GROQ_API_BASE'),
    openai_api_key=os.getenv('GROQ_API_KEY'),
    model_name="llama3-8b-8192",
    temperature=0,
    streaming=True,
)

In [3]:
POST_URL = "https://blog.nomic.ai/posts/nomic-embed-text-v1"

In [4]:
from langchain_community.document_loaders import WebBaseLoader

docs = WebBaseLoader(POST_URL).load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
len(docs)

1

In [6]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=7500, chunk_overlap=100
)
doc_splits = text_splitter.split_documents(docs)

In [8]:
import os

from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough


ImportError: cannot import name 'AliasChoices' from 'pydantic' (/Users/pangmengting/venv/lib/python3.10/site-packages/pydantic/__init__.cpython-310-darwin.so)

In [19]:
from langchain_nomic import NomicEmbeddings
from langchain_nomic.embeddings import NomicEmbeddings
vectorstore = Chroma.from_documents(
    persist_directory='/Users/pangmengting/Documents/workspace/python-learning/data/chroma_vector_db',
    documents=doc_splits,
    embedding=NomicEmbeddings(model="nomic-embed-text-v1"),
)
retriever = vectorstore.as_retriever()

ImportError: cannot import name 'AliasChoices' from 'pydantic' (/Users/pangmengting/venv/lib/python3.10/site-packages/pydantic/__init__.cpython-310-darwin.so)

In [ ]:
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | cf_llm_openai
        | StrOutputParser()
)


In [ ]:
response = chain.invoke("How did Nomic Embed get trained?")
response

In [12]:
cf_llm_openai.invoke("nihao")

AIMessage(content='Nǐ hǎo! (Nice to meet you too!) How can I assist you today?', id='run-04bb9c8a-dd7f-47d1-8130-ad3acd9313ba-0')

In [17]:
!pip show pydantic


Name: pydantic
Version: 1.10.17
Summary: Data validation and settings management using python type hints
Home-page: https://github.com/pydantic/pydantic
Author: Samuel Colvin
Author-email: s@muelcolvin.com
License: MIT
Location: /Users/pangmengting/venv/lib/python3.10/site-packages
Requires: typing-extensions
Required-by: anthropic, chainlit, chromadb, cloudflare, docarray, fastapi, google-generativeai, gotrue, kor, langchain, langchain-app, langchain-core, langchainplus-sdk, langserve, langsmith, literalai, llamaindex-py-client, nomic, openai, openapi-schema-pydantic, pandasai, postgrest, pyautogen, qdrant-client, qianfan, sqlmodel, zhipuai


In [16]:
!pip install --upgrade pydantic==1.10.17


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 3.5 MB/s eta 0:00:00a 0:00:010m
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.8.2
    Uninstalling pydantic-2.8.2:
      Successfully uninstalled pydantic-2.8.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anthropic 0.3.6 requires anyio<4,>=3.5.0, but you have anyio 4.4.0 which is incompatible.
chainlit 1.1.306 requires packaging<24.0,>=23.1, but you have packaging 24.1 which is incompatible.
chainlit 1.1.306 requires uvicorn<0.26.0,>=0.25.0, but you have uvicorn 0.23.2 which is incompatible.
sqlmodel 0.0.8 requires SQLAlchemy<=1.4.41,>=1.4.17, but you have sqlalchemy 2.0.31 which is incompatible.
zhipuai 2.0.1 requires pydantic>=2.5.2, but you have pydantic 1.10.17 which is incompatible.

[notice] A new release